In [50]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("TV.csv")
df.shape

(666, 11)

풀이시간에 질문은 공개 채팅으로~

### Q1.

In [3]:
df_q1 = df[["Frequency", "Picture_quality", "Speaker"]].copy()
df_q1.head(2)

,Frequency,Picture_quality,Speaker
0,1 Year Warranty,60 Hz Refresh Rate,2 x HDMI | 2 x USB
1,60 Hz Refresh Rate,HD Ready 1366 x 768 Pixels,20 W Speaker Output


In [6]:
cnt_1 = sum(df_q1["Frequency"] == "60 Hz Refresh Rate")
cnt_2 = sum(df_q1["Picture_quality"] == "60 Hz Refresh Rate")
cnt_3 = sum(df_q1["Speaker"] == "60 Hz Refresh Rate")
cnt_1 + cnt_2 + cnt_3

509

In [23]:
# ser_u = pd.Series(df_q1["Frequency"].unique())
ser_u = pd.Series(df_q1["Picture_quality"].unique())
# ser_u = pd.Series(df_q1["Speaker"].unique())
# ser_u[ser_u.str.contains("Hz")]
# ser_u[ser_u.str.contains("[0-9]{2,3} Hz")]
ser_u[ser_u.str.contains("^[0-9]{2,3} Hz|[^0-9][0-9]{2,3} Hz")]

0                                    60 Hz Refresh Rate
15                                  120 Hz Refresh Rate
16                                  100 Hz Refresh Rate
18                                  200 Hz Refresh Rate
20                                   58 Hz Refresh Rate
22                                   50 Hz Refresh Rate
48    60 Hz Refresh Rate for a blur-free picture qua...
61                                  300 Hz Refresh Rate
dtype: object

In [16]:
# pd.Series([True, False]).astype("int")
pd.Series([True, False]) + 0

0    1
1    0
dtype: int32

In [14]:
df_q1["hz_1"] = df_q1["Frequency"].str.contains("Hz") + 0
df_q1["hz_2"] = df_q1["Picture_quality"].str.contains("Hz") + 0
df_q1["hz_3"] = df_q1["Speaker"].str.contains("Hz") + 0
df_q1["hz_cnt"] = df_q1["hz_1"] + df_q1["hz_2"] + df_q1["hz_3"]

In [15]:
df_q1.head(2)

,Frequency,Picture_quality,Speaker,hz_1,hz_2,hz_3,hz_cnt
0,1 Year Warranty,60 Hz Refresh Rate,2 x HDMI | 2 x USB,0,1,0,1
1,60 Hz Refresh Rate,HD Ready 1366 x 768 Pixels,20 W Speaker Output,1,0,0,1


In [17]:
df_q1["hz_cnt"].value_counts()

hz_cnt
1    662
0      4
Name: count, dtype: int64

In [19]:
df_q1_sub = df_q1.loc[df_q1["hz_cnt"] > 0, ]
len(df_q1), len(df_q1_sub)

(666, 662)

In [22]:
df_q1_sub.iloc[:, :3].apply(lambda x: x.str.contains("60 Hz").sum(), 
                            axis = 1).value_counts()

1    510
0    152
Name: count, dtype: int64

In [24]:
df_q1_sub.iloc[:, :3].apply(lambda x: x.str.contains("^60 Hz|[^0-9]60 Hz").sum(), 
                            axis = 1).value_counts()

1    510
0    152
Name: count, dtype: int64

### Q2.

In [25]:
df_q2 = df[["Stars", "Operating_system", "channel", "Picture_quality"]].copy()
df_q2.head(2)

,Stars,Operating_system,channel,Picture_quality
0,4.2,20 Speaker Output,HD Ready 1366 x 768 Pixels,60 Hz Refresh Rate
1,3.8,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels


In [ ]:
# ser_u = pd.Series(df_q2["Operating_system"].unique())
# ser_u = pd.Series(df_q2["channel"].unique())
ser_u = pd.Series(df_q2["Picture_quality"].unique())
ser_u[ser_u.str.contains("4K|8K")]

In [32]:
df_q2["cnt_4k"] = df_q2.iloc[:, 1:4].apply(lambda x: x.str.contains("4K").sum(), axis = 1)
df_q2["cnt_8k"] = df_q2.iloc[:, 1:4].apply(lambda x: x.str.contains("8K").sum(), axis = 1)

In [34]:
df_q2.tail(3)

,Stars,Operating_system,channel,Picture_quality,cnt_4k,cnt_8k
663,0.0,20 Speaker Output,HD Ready 1366 x 786 Pixels,60 Hz Refresh Rate,0,0
664,0.0,Full HD 1920 x 1080 Pixels,Netflix|Prime Video|Apple TV|Disney+Hotstar|Yo...,16 Speaker Output,0,0
665,4.4,10W + 10W Speaker Output,Ultra HD (4K) 3840 x 2160 pixels Pixels,50 Hz Refresh Rate,1,0


In [35]:
df_q2["cnt_4k"].value_counts()

cnt_4k
1    365
0    301
Name: count, dtype: int64

In [36]:
stat_4k = df_q2.loc[df_q2["cnt_4k"] >= 1, "Stars"].mean()
stat_8k = df_q2.loc[df_q2["cnt_8k"] >= 1, "Stars"].mean()
stat_4k, stat_8k

(3.2180821917808218, 3.6)

In [37]:
round(abs(stat_4k - stat_8k), 2)

0.38

### Q3.

In [41]:
df_q3 = df.loc[~df["channel"].str.contains("Pixel|Oper"), ].reset_index()
# df_q3.head(1)

In [45]:
df_q3["x1"] = df_q3["Reviews"] / df_q3["Ratings"]
df_q3["x2"] = df_q3["MRP"]
df_q3["x3"] = df_q3["current_price"] / df_q3["MRP"]
df_q3["x4"] = df_q3["channel"].str.contains("Netflix") + 0
df_q3["x5"] = df_q3["channel"].str.contains("Prime Video") + 0
df_q3["x6"] = df_q3["Picture_quality"].str.contains("4K|8K") + 0

In [ ]:
df_model = df_q3[["Stars", "x1", "x2", "x3", "x4", "x5", "x6"]]
df_model.head(2)

In [ ]:
len(df_model)

In [ ]:
df_model.isna().sum()

In [ ]:
df_model = df_model.dropna()
len(df_model)

In [ ]:
model_rf = RandomForestRegressor(random_state = 123)
model_rf.fit(X = df_model.drop(columns = "Stars"),
             y = df_model["Stars"])

In [53]:
model_rf.feature_importances_

array([0.31220617, 0.17383963, 0.4446912 , 0.02714087, 0.02400442,
       0.01811773])

In [54]:
pd.Series(model_rf.feature_importances_, index = df_model.columns[1:]).reset_index()

,index,0
0,x1,0.312206
1,x2,0.173840
2,x3,0.444691
3,x4,0.027141
4,x5,0.024004
5,x6,0.018118


버전 차이에 의해 답이 다를 수 있음.

In [55]:
pd.__version__

'2.1.1'

In [56]:
import sklearn
sklearn.__version__

'1.2.2'